In [2]:
!pip3 install pymupdf pandas


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip3 install --upgrade pip


In [3]:
import fitz  # PyMuPDF
from typing import List, Tuple, Dict
import pandas as pd

In [4]:
# https://www.navercorp.com/investment/irReport

## pdf 파일을 열어보기
pdf_path = "네이버_사업보고서_2023.pdf"

doc = fitz.open(pdf_path)
content = []
print(doc)

Document('네이버_사업보고서_2023.pdf')


In [5]:
## document는 각각의 페이지로 구성된 리스트

## 아래를 보면 페이지 수랑 일치함
print(len(doc))

## 첫페이지는 이렇게 접근 가능함
print(doc[0])  

565
page 0 of 네이버_사업보고서_2023.pdf


In [6]:
## 페이지의 정보를 dict 형태로 반환
doc[0].get_text("dict")

## 기본 페이지정보 (너비(width),높이(height) 너비, 블럭) 등이 있음
## 1개의 블럭은 1개의 사각형 영역을 의미


{'width': 595.0,
 'height': 842.0,
 'blocks': [{'number': 0,
   'type': 0,
   'bbox': (236.2100067138672, 60.265625, 358.7860412597656, 76.265625),
   'lines': [{'spans': [{'size': 16.0,
       'flags': 4,
       'font': 'Batang',
       'color': 0,
       'ascender': 0.8583984375,
       'descender': -0.1416015625,
       'text': '목                 차',
       'origin': (236.2100067138672, 74.0),
       'bbox': (236.2100067138672, 60.265625, 358.7860412597656, 76.265625)}],
     'wmode': 0,
     'dir': (1.0, 0.0),
     'bbox': (236.2100067138672, 60.265625, 358.7860412597656, 76.265625)}]},
  {'number': 1,
   'type': 0,
   'bbox': (66.0, 89.416015625, 545.0070190429688, 99.416015625),
   'lines': [{'spans': [{'size': 10.0,
       'flags': 4,
       'font': 'Batang',
       'color': 0,
       'ascender': 0.8583984375,
       'descender': -0.1416015625,
       'text': '사 업 보 고 서...............................................................................................................

In [7]:
## 기본적으로 1개의 페이지는 여러 블럭으로 나눠어져 있음

## 회사의 개요 페이지는 33개 블럭으로 되어있네
len(doc[5].get_text("dict")["blocks"])

33

In [8]:

doc[5].get_text("dict")["blocks"][0]

{'number': 0,
 'type': 0,
 'bbox': (240.9499969482422, 61.548828125, 354.0439758300781, 79.548828125),
 'lines': [{'spans': [{'size': 18.0,
     'flags': 4,
     'font': 'Batang',
     'color': 255,
     'ascender': 0.8583984375,
     'descender': -0.1416015625,
     'text': 'I. 회사의 개요',
     'origin': (240.9499969482422, 77.0),
     'bbox': (240.9499969482422,
      61.548828125,
      354.0439758300781,
      79.548828125)}],
   'wmode': 0,
   'dir': (1.0, 0.0),
   'bbox': (240.9499969482422,
    61.548828125,
    354.0439758300781,
    79.548828125)}]}

블럭정보에는 
- number(블럭번호, 0이면 페이지의 첫번째 블럭)
- type (블럭타입, 0이면 텍스트)
- bbox : 블럭의 경계좌표
- lines : 행 (여러 줄로 이루어진 문단이면 여러개가 있음)
블럭 1개를 자세히 보면 각각의 줄 (line)로 이루어져 있음

1개의 줄에 그 줄의 텍스트 정보가 나와있음
 - text : 실제로 우리가 관심이 있는 텍스트
- font : 글꼴
- size : 글자크기

{'number': 0,  ## 블럭번호
 'type': 0, ## 0이면 텍스트블럭을 의미
 'bbox': (240.9499969482422, 61.548828125, 354.0439758300781, 79.548828125),
 'lines': [{'spans': [{'size': 18.0,
     'flags': 4,
     'font': 'Batang',
     'color': 255,
     'ascender': 0.8583984375,
     'descender': -0.1416015625,
     'text': 'I. 회사의 개요',
     'origin': (240.9499969482422, 77.0),
     'bbox': (240.9499969482422,
      61.548828125,
      354.0439758300781,
      79.548828125)}],
   'wmode': 0,
   'dir': (1.0, 0.0),
   'bbox': (240.9499969482422,
    61.548828125,
    354.0439758300781,
    79.548828125)}]}

    (x0, y0)                              (x1, y0)
  +--------------------------------------+
  |                                      |
  |                                      |
  |                                      |
  +--------------------------------------+
(x0, y1)                              (x1, y1)

In [9]:
## 블럭 1개를 보기
doc[5].get_text("dict")["blocks"][1]

{'number': 1,
 'type': 0,
 'bbox': (50.0, 87.265625, 154.83200073242188, 103.265625),
 'lines': [{'spans': [{'size': 16.0,
     'flags': 4,
     'font': 'Batang',
     'color': 255,
     'ascender': 0.8583984375,
     'descender': -0.1416015625,
     'text': '1. 회사의 개요',
     'origin': (50.0, 101.0),
     'bbox': (50.0, 87.265625, 154.83200073242188, 103.265625)}],
   'wmode': 0,
   'dir': (1.0, 0.0),
   'bbox': (50.0, 87.265625, 154.83200073242188, 103.265625)}]}

In [10]:
doc[5].get_text("dict")["blocks"][2]

{'number': 2,
 'type': 0,
 'bbox': (50.0, 114.69921875, 535.6040649414062, 192.58799743652344),
 'lines': [{'spans': [{'size': 12.0,
     'flags': 4,
     'font': 'Batang',
     'color': 0,
     'ascender': 0.8583984375,
     'descender': -0.1416015625,
     'text': ' ',
     'origin': (50.0, 125.0),
     'bbox': (50.0, 114.69921875, 53.99609375, 126.69921875)}],
   'wmode': 0,
   'dir': (1.0, 0.0),
   'bbox': (50.0, 114.69921875, 53.99609375, 126.69921875)},
  {'spans': [{'size': 12.0,
     'flags': 4,
     'font': 'Batang',
     'color': 0,
     'ascender': 0.8583984375,
     'descender': -0.1416015625,
     'text': '가. 연결대상 종속회사 현황(요약)',
     'origin': (50.0, 141.0),
     'bbox': (50.0, 130.69921875, 230.51544189453125, 142.69921875)}],
   'wmode': 0,
   'dir': (1.0, 0.0),
   'bbox': (50.0, 130.69921875, 230.51544189453125, 142.69921875)},
  {'spans': [{'size': 12.0,
     'flags': 4,
     'font': 'Batang',
     'color': 0,
     'ascender': 0.8583984375,
     'descender': -0.14160156

In [11]:
## 그래서 우리는 pdf 문서안의 모든 페이지, 모든 블럭, 모든 줄을 돌면서
## 모든 텍스트와 폰트크기를 같이 구해서 리스트로 만들어줄 것임

## 일단 모든 텍스트를 텍스트 / 폰트크기 조합으로 추출해보기
TextBlock = Tuple[str, float]

def extract_texts_with_font_size(pdf_path: str) -> List[TextBlock]:
    doc = fitz.open(pdf_path)
    content = []
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            lines = block.get("lines", [])

            for line in lines:
                for span in line["spans"]:
                    text = span["text"]
                    size = span["size"]
                    content.append((text, size))
    
    return content
result = extract_texts_with_font_size(pdf_path)


[('목                 차', 16.0), ('사 업 보 고 서............................................................................................................................................1', 10.0), ('【 대표이사 등의 확인 】...........................................................................................................................2', 10.0), ('I. 회사의 개요............................................................................................................................................3', 10.0), ('1. 회사의 개요......................................................................................................................................3', 10.0), ('2. 회사의 연혁......................................................................................................................................7', 10.0), ('3. 자본금 변동사항.............................................................................................................................10', 10.0), ('4. 주식의 총수 등...................

In [13]:
for i in result[1000:1100]:
  print(i)

('5. 정관에 관한 사항', 16.0)
(' ', 12.0)
('가. 정관변경 이력', 12.0)
(' ', 12.0)
('당사는 2021년 3월 24일에 개최한 제22기 정기주주총회에서 정관이 최신 개정되었으며, 공', 12.0)
('시대상기간(2021년~2023년)중 정관 변경 이력은 다음과 같습니다.', 12.0)
(' ', 12.0)
(' ', 12.0)
('나. 사업목적 현황', 12.0)
(' ', 12.0)
('정관변경일', 8.0)
('해당주총명', 8.0)
('주요변경사항', 8.0)
('변경이유', 8.0)
('2021년 03월 24일', 8.0)
('제22기 정기주주총회', 8.0)
('제3조(본점의소재지및 지점 등의 설치)', 8.0)
('제15조(명의개서대리인)', 8.0)
('제16조(주주명부 작성 비치)', 8.0)
('제17조(주주명부의 폐쇄 및 기준일)', 8.0)
('제37조(이사의 임기)', 8.0)
('부칙', 8.0)
('- 본점 소재지 최소 행정 단위로 수정', 8.0)
('- 전자등록제도 도입에 따른 변동사항 반영', 8.0)
('- 관련 법규에 의한 근거규정 신설 등', 8.0)
('- 전자등록제도 도입에 따른 변동사항 반영', 8.0)
('- 관련 상법규정 준용', 8.0)
('- 정관 시행일자 추가', 8.0)
('※ 당사는 공시서류 제출일 이후인 2024년 3월 26일에 제25기 정기주주총회를 개최할 예정이며, 해당 주주총회 안건으로', 9.0)
('정관변경', 9.0)
("    안건이 상정되어 있습니다. 안건의 세부 내용은 당사가 2024년 2월 26일에 공시한 '주주총회소집공고' 공시를 참조해", 9.0)
('주시기', 9.0)
('    바라며, 안건이 부결되거나 수정이 발생할 경우 정정보고서를 통해 그 내용 및 사유 등을 반영할 예정 입니다.', 9.0)
('구 분', 10.0)
('사업목적', 10.0)
('사업영위 여부', 10.0)
('1', 10.0)
('컴퓨터 및 통신기기를 이용한 정보자료처리

In [14]:
## 소제목이 되는 텍스트의 글자 크기를 확인해보기 => 16인걸 확인했음!

some_titles = ["2. 주요 제품 및 서비스", "3. 원재료 및 생산설비"]
title_font_sizes = []

for text, size in result:
        if text in some_titles:
            title_font_sizes.append((text, size))
print(title_font_sizes)

[('2. 주요 제품 및 서비스', 16.0), ('3. 원재료 및 생산설비', 16.0)]


In [17]:
## 소제목 기준으로 텍스트들을 쪼개기

TextBlock = Tuple[str, float]

def create_chunks_based_on_font_size(content: List[TextBlock], target_font_size: float) -> List[List[TextBlock]]:
    chunks = []  # 텍스트 블록의 청크를 저장할 리스트
    current_chunk = []  # 현재 청크를 저장할 임시 리스트

    for text_block in content:
        text, font_size = text_block

        if font_size == target_font_size:
            # 대상 글꼴 크기를 가진 텍스트 블록을 찾았습니다.
            # 이는 새로운 청크의 시작을 의미합니다.

            # 현재 청크에 텍스트 블록이 있으면 저장합니다.
            if current_chunk:
                chunks.append(current_chunk)
            
            # 이 텍스트 블록으로 새로운 청크를 시작합니다.
            current_chunk = [text_block]
        else:
            # 이 텍스트 블록은 대상 글꼴 크기를 가지고 있지 않습니다.
            # 현재 청크에 추가합니다.
            current_chunk.append(text_block)

    # 마지막 청크가 비어 있지 않으면 추가합니다.
    if current_chunk:
        chunks.append(current_chunk)

    return chunks

# 테스트용 데이터
content = [
    ("제목 1", 12.0),
    ("내용 1 아래 내용입니다.", 10.0),
    ("제목 2", 12.0),
    ("제목 2 아래 내용입니다.", 10.0),
    ("제목 2 아래 추가 내용입니다.", 10.0),
    ("제목 3", 12.0),
    ("제목 3 아래 내용입니다.", 10.0)
]
target_font_size = 12.0
chunks = create_chunks_based_on_font_size(content, target_font_size)
for i, chunk in enumerate(chunks):
    print(f"청크 {i+1}:")
    for text, size in chunk:
        print(f"  {text} ({size})")

청크 1:
  제목 1 (12.0)
  내용 1 아래 내용입니다. (10.0)
청크 2:
  제목 2 (12.0)
  제목 2 아래 내용입니다. (10.0)
  제목 2 아래 추가 내용입니다. (10.0)
청크 3:
  제목 3 (12.0)
  제목 3 아래 내용입니다. (10.0)


In [18]:
## 그대로 적용해보기

chunks = create_chunks_based_on_font_size(result, 16)
len(chunks)
# for i, chunk in enumerate(chunks):
#     print(f"청크 {i+1}:")
#     for text, size in chunk:
#         print(f"  {text} ({size})")

44

In [19]:

## 이 중에서 특정 제목 기준으로 필터링하기

def filter_chunks_by_text_list(chunks: List[List[TextBlock]], delimiter_texts: List[str]) -> List[List[TextBlock]]:
    filtered_chunks = []
    
    for chunk in chunks:
        # Check if any text in the chunk matches any of the delimiter texts
        if any(text in delimiter_texts for text, _ in chunk):
            filtered_chunks.append(chunk)
    
    return filtered_chunks

# 특정 텍스트 목록을 포함하는 청크를 필터링합니다.
delimiter_texts = ["2. 주요 제품 및 서비스", "3. 원재료 및 생산설비"]
filtered_chunks = filter_chunks_by_text_list(chunks, delimiter_texts)

# 필터링된 청크를 출력합니다.
for i, chunk in enumerate(filtered_chunks):
    print(f"필터링된 청크 {i+1}:")
    for text, size in chunk:
        print(f"  {text} ({size})")

필터링된 청크 1:
  2. 주요 제품 및 서비스 (16.0)
    (12.0)
  네이버 및 종속회사의 서비스는 서치플랫폼, 커머스, 핀테크, 콘텐츠, 클라우드로 구성되어 (12.0)
  있으며, 각 사업부문별 주요 서비스는 다음과 같습니다. (12.0)
    (12.0)
    (12.0)
    (12.0)
  ① 서치플랫폼 (12.0)
    (12.0)
  서치플랫폼은 검색과 디스플레이 광고로 구성되어 있습니다. 검색은 이용자의 정보탐색 수 (12.0)
  요를 네이버를 통해 연결시켜 비즈니스 활동을 촉진시키는 사업으로, 지속적인 상품 개선과 (12.0)
  카테고리 확대, AI 기술을 활용한 검색 고도화를 통해 지속적으로 성장하고 있습니다. 디스 (12.0)
  플레이는 광고주가 원하는 상업적 메시지를 사용자에게 노출시켜주는 상품을 제공하는 사업 (12.0)
  으로, 디스플레이 플랫폼 고도화와 성과형 광고상품, 진화하는 광고주 수요에 대응하여 동영 (12.0)
  상/프리미엄 상품 출시 등 다양한 신규 상품 출시를 통해 광고 효과를 제고하는 등의 노력을 (12.0)
  펼치고 있습니다. (12.0)
    (12.0)
    (12.0)
    (12.0)
  ② 커머스 (12.0)
    (12.0)
  커머스는 커머스 광고(쇼핑 관련 검색과 디스플레이 광고), 중개 및 판매(수수료), 멤버십 매 (12.0)
  출로 구성되어 있습니다. 커머스는 브랜드, SME를 포함한 다양한 판매자들과 함께 성장하며 (12.0)
  , 이용자에게 차별화된 쇼핑 경험을 제공하기 위해 노력 중입니다. 브랜드스토어, 쇼핑라이브 (12.0)
  , 장보기, C2C 플랫폼(KREAM, Poshmark) 등 신규 버티컬 커머스 서비스 강화, 여행(항공, (12.0)
  호텔, 투어) 및 네이버플레이스 내 예약 서비스 제공,  데이터 기반 풀필먼트 플랫폼 제공, (12.0)
  CJ대한통운 및 신세계/이마트 제휴 등을 통해 이용자와 판매자의 다양한 니즈를 대응하며 커 

In [20]:
## 주석처리된 특정 텍스트들은 다시한번 지워주기
## 보통 주석은 같은 글자 크기인 경우가 많음
##   전자공시시스템 dart.fss.or.kr (9.0)

def exclude_chunks_based_on_font_size(chunks: List[List[TextBlock]], exclude_font_size: float) -> List[List[TextBlock]]:
    """특정 글꼴 크기에 따라 청크를 제외합니다."""
    filtered_chunks = []
    
    for chunk in chunks:
        temp_chunk = []
        for text, size in chunk:
            # exclude_font_size와 일치하는 글꼴 크기를 가진 텍스트 블록을 건너뜁니다.
            if size == exclude_font_size:
                continue
            temp_chunk.append((text, size))
        
        # 임시 청크가 비어 있지 않으면 필터링된 청크에 추가합니다.
        if temp_chunk:
            filtered_chunks.append(temp_chunk)
    
    return filtered_chunks


filtered_chunks_2 = exclude_chunks_based_on_font_size(filtered_chunks,9)

# 필터링된 청크를 출력합니다.
for i, chunk in enumerate(filtered_chunks_2):
    print(f"필터링된 청크 {i+1}:")
    for text, size in chunk:
        print(f"  {text} ({size})")

필터링된 청크 1:
  2. 주요 제품 및 서비스 (16.0)
    (12.0)
  네이버 및 종속회사의 서비스는 서치플랫폼, 커머스, 핀테크, 콘텐츠, 클라우드로 구성되어 (12.0)
  있으며, 각 사업부문별 주요 서비스는 다음과 같습니다. (12.0)
    (12.0)
    (12.0)
    (12.0)
  ① 서치플랫폼 (12.0)
    (12.0)
  서치플랫폼은 검색과 디스플레이 광고로 구성되어 있습니다. 검색은 이용자의 정보탐색 수 (12.0)
  요를 네이버를 통해 연결시켜 비즈니스 활동을 촉진시키는 사업으로, 지속적인 상품 개선과 (12.0)
  카테고리 확대, AI 기술을 활용한 검색 고도화를 통해 지속적으로 성장하고 있습니다. 디스 (12.0)
  플레이는 광고주가 원하는 상업적 메시지를 사용자에게 노출시켜주는 상품을 제공하는 사업 (12.0)
  으로, 디스플레이 플랫폼 고도화와 성과형 광고상품, 진화하는 광고주 수요에 대응하여 동영 (12.0)
  상/프리미엄 상품 출시 등 다양한 신규 상품 출시를 통해 광고 효과를 제고하는 등의 노력을 (12.0)
  펼치고 있습니다. (12.0)
    (12.0)
    (12.0)
    (12.0)
  ② 커머스 (12.0)
    (12.0)
  커머스는 커머스 광고(쇼핑 관련 검색과 디스플레이 광고), 중개 및 판매(수수료), 멤버십 매 (12.0)
  출로 구성되어 있습니다. 커머스는 브랜드, SME를 포함한 다양한 판매자들과 함께 성장하며 (12.0)
  , 이용자에게 차별화된 쇼핑 경험을 제공하기 위해 노력 중입니다. 브랜드스토어, 쇼핑라이브 (12.0)
  , 장보기, C2C 플랫폼(KREAM, Poshmark) 등 신규 버티컬 커머스 서비스 강화, 여행(항공, (12.0)
  호텔, 투어) 및 네이버플레이스 내 예약 서비스 제공,  데이터 기반 풀필먼트 플랫폼 제공, (12.0)
  CJ대한통운 및 신세계/이마트 제휴 등을 통해 이용자와 판매자의 다양한 니즈를 대응하며 커 